# 02 - Data Processing

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

## Imports

In [2]:
import numpy as np
import pandas as pd

import stock_utils.constants as cnst
import stock_utils.data as sd

pd.set_option('display.max_columns', None)

NSE_DATA_DIR = PosixPath('../data/NSE') | Valid: True
SCREENER_EPS_DATA_DIR = PosixPath('../data/Screener.in/EPS') | Valid: True
PROCESSED_DATA_DIR = PosixPath('../data/processed') | Valid: True


## Constants

In [3]:
stock_symbols = sd.get_all_stock_symbols(
    cnst.NSE_DATA_DIR
)

stock_symbols

['DEEPAKFERT', 'HDFCBANK', 'INDIGOPNTS', 'ITBEES', 'JUBLFOOD', 'NH']

In [4]:
STOCK_SYMBOL = stock_symbols[1]
STOCK_SYMBOL

'HDFCBANK'

## Data loading

In [5]:
stock_df = pd.read_parquet(cnst.NSE_DATA_DIR.joinpath(STOCK_SYMBOL, "consolidated.parquet"))
stock_df.columns = [
    'Date', 'Series',
    'Open', 'High', 'Low', 'Prev Close', 'LTP', 'Close',
    'VWAP', '52W H', '52W L',
    'Volume', 'Value', '#Trades'
] + (["PE"] if "PE" in stock_df.columns else [])
stock_df

,Date,Series,Open,High,Low,Prev Close,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE
0,2020-01-01,EQ,1276.10,1280.00,1270.60,1272.10,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625,31.185
1,2020-01-02,EQ,1279.00,1288.00,1279.00,1278.60,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570,31.384
2,2020-01-03,EQ,1282.20,1285.00,1263.60,1286.75,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066,30.937
3,2020-01-06,EQ,1260.00,1261.80,1236.00,1268.40,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007,30.267
4,2020-01-07,EQ,1258.90,1271.45,1252.25,1240.95,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026,30.746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,2024-06-24,EQ,1647.00,1675.10,1645.75,1665.75,1671.00,1672.40,1664.93,1757.5,1363.55,14272497,2.376264e+10,387580,20.286
1128,2024-06-25,EQ,1671.10,1716.95,1671.10,1672.40,1707.00,1711.35,1705.04,1757.5,1363.55,37260774,6.353113e+10,690683,20.759
1129,2024-06-26,EQ,1707.95,1710.90,1691.10,1711.35,1693.50,1701.50,1703.01,1757.5,1363.55,23798333,4.052887e+10,424591,20.639
1130,2024-06-27,EQ,1696.10,1713.00,1686.60,1701.50,1696.45,1696.15,1697.92,1757.5,1363.55,27537325,4.675614e+10,586504,20.574


In [6]:
stock_df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132 entries, 0 to 1131
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        1132 non-null   datetime64[ns]
 1   Series      1132 non-null   object        
 2   Open        1132 non-null   float64       
 3   High        1132 non-null   float64       
 4   Low         1132 non-null   float64       
 5   Prev Close  1132 non-null   float64       
 6   LTP         1132 non-null   float64       
 7   Close       1132 non-null   float64       
 8   VWAP        1132 non-null   float64       
 9   52W H       1132 non-null   float64       
 10  52W L       1132 non-null   float64       
 11  Volume      1132 non-null   int64         
 12  Value       1132 non-null   float64       
 13  #Trades     1132 non-null   int64         
 14  PE          1132 non-null   float64       
dtypes: datetime64[ns](1), float64(11), int64(2), object(1)
memory usage: 189

## Feature engineering

### Daily candle based columns

In [7]:
stock_df['Range'] = stock_df['High'] - stock_df['Low']
stock_df['IsGreen'] = (stock_df['Close'] >= stock_df['Prev Close']).astype(int)
stock_df['Is52WLow'] = np.isclose(stock_df['Low'], stock_df['52W L'], atol = 0, rtol = 1e-4).astype(int)
stock_df['Is52WHigh'] = np.isclose(stock_df['High'], stock_df['52W H'], atol = 0, rtol = 1e-4).astype(int)
stock_df = stock_df.drop(columns = ['Series', 'Prev Close'])
stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh
175,2020-08-21,1065.40,1088.80,1064.60,1085.70,1085.65,1081.52,2288.8,738.75,14462237,1.564118e+10,228261,21.844,24.20,1,0,0
451,2021-09-30,1586.00,1606.35,1583.10,1592.85,1594.95,1594.87,1641.0,1042.65,7987674,1.273928e+10,216914,27.623,23.25,1,0,0
649,2022-07-18,1348.00,1366.10,1342.05,1348.20,1347.55,1347.82,1725.0,1271.60,10166110,1.370213e+10,291810,19.638,24.05,0,0,0
106,2020-05-15,890.50,892.40,875.00,888.00,888.15,885.00,2503.3,738.75,12174870,1.077480e+10,280499,17.870,17.40,0,0,0
1022,2024-01-18,1494.00,1515.00,1480.05,1490.00,1486.15,1495.03,1757.5,1460.25,80535465,1.204027e+11,1582497,18.027,34.95,0,0,0
583,2022-04-12,1484.00,1506.85,1480.15,1494.90,1493.50,1492.32,1725.0,1292.00,13717701,2.047121e+10,365154,21.765,26.70,0,0,0
333,2021-04-09,1426.00,1432.80,1415.10,1423.50,1421.75,1422.88,1641.0,826.10,14078908,2.003262e+10,214798,24.623,17.70,0,0,0
557,2022-03-04,1360.00,1386.00,1336.90,1364.95,1366.50,1359.66,1725.0,1336.90,10609682,1.442558e+10,351310,23.666,49.10,0,1,0
276,2021-01-14,1471.15,1488.00,1456.00,1474.00,1468.75,1468.80,1496.9,738.75,6148583,9.031062e+09,187531,29.552,32.00,0,0,0
270,2021-01-06,1435.00,1440.00,1413.10,1419.80,1420.55,1423.55,1464.4,738.75,11067025,1.575451e+10,151854,28.582,26.90,0,0,0


In [8]:
((stock_df[['IsGreen', 'Is52WLow', 'Is52WHigh']].sum() * 100) / len(stock_df)).round(2)

IsGreen      52.21
Is52WLow      1.59
Is52WHigh     2.30
dtype: float64

In [9]:
stock_df.shape

(1132, 17)

### Streak

In [10]:
stock_df["Streak"] = (stock_df["IsGreen"] != stock_df["IsGreen"].shift(1)).cumsum()
stock_df["Streak"] = stock_df.groupby("Streak").cumcount() + 1
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh,Streak
0,2020-01-01,1276.10,1280.00,1270.60,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625,31.185,9.40,1,0,0,1
1,2020-01-02,1279.00,1288.00,1279.00,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570,31.384,9.00,1,0,0,2
2,2020-01-03,1282.20,1285.00,1263.60,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066,30.937,21.40,0,0,0,1
3,2020-01-06,1260.00,1261.80,1236.00,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007,30.267,25.80,0,0,0,2
4,2020-01-07,1258.90,1271.45,1252.25,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026,30.746,19.20,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,2024-06-24,1647.00,1675.10,1645.75,1671.00,1672.40,1664.93,1757.5,1363.55,14272497,2.376264e+10,387580,20.286,29.35,1,0,0,1
1128,2024-06-25,1671.10,1716.95,1671.10,1707.00,1711.35,1705.04,1757.5,1363.55,37260774,6.353113e+10,690683,20.759,45.85,1,0,0,2
1129,2024-06-26,1707.95,1710.90,1691.10,1693.50,1701.50,1703.01,1757.5,1363.55,23798333,4.052887e+10,424591,20.639,19.80,0,0,0,1
1130,2024-06-27,1696.10,1713.00,1686.60,1696.45,1696.15,1697.92,1757.5,1363.55,27537325,4.675614e+10,586504,20.574,26.40,0,0,0,2


### Date based columns

In [11]:
stock_df['Day'] = stock_df['Date'].dt.day
stock_df['Month'] = stock_df['Date'].dt.month
stock_df['Year'] = stock_df['Date'].dt.year
stock_df['Weekday'] = stock_df['Date'].dt.weekday
stock_df['DayOfYear'] = stock_df['Date'].dt.day_of_year
stock_df['Quarter'] = stock_df['Date'].dt.quarter
stock_df['DaysSinceLastTradingSession'] = (stock_df['Date'] - stock_df['Date'].shift(1)).dt.days.fillna(1).astype(int)

stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession
321,2021-03-22,1494.90,1494.90,1460.40,1474.00,1469.15,1473.27,1641.00,738.75,7007816,1.032442e+10,257662,29.560,34.50,0,0,0,1,22,3,2021,0,81,1,3
867,2023-06-06,1607.65,1609.85,1592.25,1605.00,1599.40,1598.09,1734.45,1271.60,16091426,2.571562e+10,233706,19.401,17.60,0,0,0,2,6,6,2023,1,157,2,1
568,2022-03-22,1481.30,1498.00,1457.20,1496.80,1494.15,1478.67,1725.00,1292.00,5230907,7.734791e+09,217951,25.877,40.80,1,0,0,4,22,3,2022,1,81,1,1
1074,2024-04-04,1504.00,1530.00,1504.00,1529.00,1527.60,1520.65,1757.50,1363.55,44467533,6.761971e+10,638189,18.530,26.00,1,0,0,6,4,4,2024,3,95,2,1
174,2020-08-20,1054.95,1068.95,1052.00,1057.35,1059.00,1059.30,2288.80,738.75,9608591,1.017840e+10,149244,21.308,16.95,0,0,0,1,20,8,2020,3,233,3,1
1014,2024-01-08,1678.50,1682.35,1660.00,1662.90,1663.45,1671.84,1757.50,1460.25,8999235,1.504530e+10,368430,20.178,22.35,0,0,0,2,8,1,2024,0,8,1,3
892,2023-07-12,1655.35,1666.30,1627.15,1636.00,1632.95,1643.87,1757.50,1337.00,35479179,5.832320e+10,498866,19.808,39.15,0,0,0,4,12,7,2023,2,193,3,1
811,2023-03-13,1587.90,1603.80,1564.95,1571.50,1568.55,1584.50,1722.10,1271.60,13221775,2.094988e+10,164259,22.858,38.85,0,0,0,2,13,3,2023,0,72,1,3
1124,2024-06-19,1613.40,1669.90,1604.00,1657.00,1657.85,1644.44,1757.50,1363.55,45065598,7.410759e+10,709705,20.110,65.90,1,0,0,6,19,6,2024,2,171,2,1
1006,2023-12-27,1681.50,1706.50,1678.60,1702.10,1703.30,1695.87,1757.50,1460.25,13504539,2.290188e+10,321373,20.661,27.90,1,0,0,2,27,12,2023,2,361,4,1


In [12]:
stock_df.shape

(1132, 25)

### Moving averages

#### `Close`

In [13]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Close {window}MA'] = stock_df['Close'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Close.*")

,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA
0,1278.60,1278.60,1278.60,1278.60,1278.60
1,1286.75,1282.68,1282.68,1282.68,1282.68
2,1268.40,1277.92,1277.92,1277.92,1277.92
3,1240.95,1268.68,1268.68,1268.68,1268.68
4,1260.60,1267.06,1267.06,1267.06,1267.06
...,...,...,...,...,...
1127,1672.40,1635.83,1592.75,1539.24,1520.53
1128,1711.35,1654.49,1602.03,1547.77,1525.30
1129,1701.50,1669.43,1616.58,1555.79,1529.64
1130,1696.15,1682.05,1626.21,1564.38,1533.78


#### `Range`

In [14]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Range {window}MA'] = stock_df['Range'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Range.*")

,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA
0,9.40,9.40,9.40,9.40,9.40
1,9.00,9.20,9.20,9.20,9.20
2,21.40,13.27,13.27,13.27,13.27
3,25.80,16.40,16.40,16.40,16.40
4,19.20,16.96,16.96,16.96,16.96
...,...,...,...,...,...
1127,29.35,29.78,36.97,29.78,26.98
1128,45.85,34.66,36.76,30.32,27.48
1129,19.80,34.44,31.08,30.48,27.37
1130,26.40,35.21,27.69,30.56,27.48


#### `VWAP`

In [15]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'VWAP {window}MA'] = stock_df['VWAP'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "VWAP.*")

,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA
0,1276.64,1276.64,1276.64,1276.64,1276.64
1,1284.56,1280.60,1280.60,1280.60,1280.60
2,1270.48,1277.23,1277.23,1277.23,1277.23
3,1247.24,1269.73,1269.73,1269.73,1269.73
4,1261.48,1268.08,1268.08,1268.08,1268.08
...,...,...,...,...,...
1127,1664.93,1630.56,1589.88,1536.44,1519.76
1128,1705.04,1648.16,1599.02,1545.04,1524.34
1129,1703.01,1664.00,1612.23,1553.15,1528.78
1130,1697.92,1677.42,1623.84,1561.65,1532.89


#### `Volume`

In [16]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Volume {window}MA'] = stock_df['Volume'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Volume.*")

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,1836849,1836849,1836849,1836849,1836849
1,3068583,2452716,2452716,2452716,2452716
2,5427775,3444402,3444402,3444402,3444402
3,5445093,3944575,3944575,3944575,3944575
4,7362247,4628109,4628109,4628109,4628109
...,...,...,...,...,...
1127,14272497,20985154,23733574,20275657,19612526
1128,37260774,24585251,23796383,21066896,19759836
1129,23798333,26160688,22398564,21432422,19638403
1130,27537325,27779183,21513331,21701466,19634090


#### `Value`

In [17]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Value {window}MA'] = stock_df['Value'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Value.*")

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,2.345001e+09,2345000988,2345000988,2345000988,2345000988
1,3.941792e+09,3143396262,3143396262,3143396262,3143396262
2,6.895886e+09,4394226092,4394226092,4394226092,4394226092
3,6.791348e+09,4993506527,4993506527,4993506527,4993506527
4,9.287302e+09,5852265530,5852265530,5852265530,5852265530
...,...,...,...,...,...
1127,2.376264e+10,34375635764,37587604715,31296013232,29856655199
1128,6.353113e+10,40726192837,38026814031,32761367259,30237999548
1129,4.052887e+10,43611478080,36237510854,33487986023,30169390436
1130,4.675614e+10,46576916482,35208163531,34110183335,30276181845


#### `#Trades`

In [18]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'#Trades {window}MA'] = stock_df['#Trades'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "#Trades.*")

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,46625,46625,46625,46625,46625
1,104570,75597,75597,75597,75597
2,157066,102753,102753,102753,102753
3,155007,115817,115817,115817,115817
4,189026,130458,130458,130458,130458
...,...,...,...,...,...
1127,387580,418097,471846,408166,391696
1128,690683,476584,478943,415756,394555
1129,424591,491500,454808,419377,394760
1130,586504,524781,450541,425809,398354


### Target columns

In [19]:
for window in cnst.TARGET_WINDOWS:
    stock_df[f'Target {window}D'] = stock_df['Close'].shift(-window)

stock_df.filter(regex = "Target.*")

,Target 3D,Target 7D,Target 15D,Target 30D
0,1240.95,1282.70,1240.85,1240.60
1,1260.60,1286.00,1244.85,1249.00
2,1257.30,1289.50,1244.55,1241.40
3,1271.40,1284.25,1213.20,1219.35
4,1282.70,1287.65,1223.20,1217.15
...,...,...,...,...
1127,1696.15,NaN,NaN,NaN
1128,1683.80,NaN,NaN,NaN
1129,NaN,NaN,NaN,NaN
1130,NaN,NaN,NaN,NaN


### Saving processed data

In [20]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-processed.parquet'), 
    index = False
)
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,2020-01-01,1276.10,1280.00,1270.60,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625,31.185,9.40,1,0,0,1,1,1,2020,2,1,1,1,1278.60,1278.60,1278.60,1278.60,9.40,9.40,9.40,9.40,1276.64,1276.64,1276.64,1276.64,1836849,1836849,1836849,1836849,2345000988,2345000988,2345000988,2345000988,46625,46625,46625,46625,1240.95,1282.70,1240.85,1240.60
1,2020-01-02,1279.00,1288.00,1279.00,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570,31.384,9.00,1,0,0,2,2,1,2020,3,2,1,1,1282.68,1282.68,1282.68,1282.68,9.20,9.20,9.20,9.20,1280.60,1280.60,1280.60,1280.60,2452716,2452716,2452716,2452716,3143396262,3143396262,3143396262,3143396262,75597,75597,75597,75597,1260.60,1286.00,1244.85,1249.00
2,2020-01-03,1282.20,1285.00,1263.60,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066,30.937,21.40,0,0,0,1,3,1,2020,4,3,1,1,1277.92,1277.92,1277.92,1277.92,13.27,13.27,13.27,13.27,1277.23,1277.23,1277.23,1277.23,3444402,3444402,3444402,3444402,4394226092,4394226092,4394226092,4394226092,102753,102753,102753,102753,1257.30,1289.50,1244.55,1241.40
3,2020-01-06,1260.00,1261.80,1236.00,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007,30.267,25.80,0,0,0,2,6,1,2020,0,6,1,3,1268.68,1268.68,1268.68,1268.68,16.40,16.40,16.40,16.40,1269.73,1269.73,1269.73,1269.73,3944575,3944575,3944575,3944575,4993506527,4993506527,4993506527,4993506527,115817,115817,115817,115817,1271.40,1284.25,1213.20,1219.35
4,2020-01-07,1258.90,1271.45,1252.25,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026,30.746,19.20,1,0,0,1,7,1,2020,1,7,1,1,1267.06,1267.06,1267.06,1267.06,16.96,16.96,16.96,16.96,1268.08,1268.08,1268.08,1268.08,4628109,4628109,4628109,4628109,5852265530,5852265530,5852265530,5852265530,130458,130458,130458,130458,1282.70,1287.65,1223.20,1217.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,2024-06-24,1647.00,1675.10,1645.75,1671.00,1672.40,1664.93,1757.5,1363.55,14272497,2.376264e+10,387580,20.286,29.35,1,0,0,1,24,6,2024,0,176,2,3,1635.83,1592.75,1539.24,1520.53,29.78,36.97,29.78,26.98,1630.56,1589.88,1536.44,1519.76,20985154,23733574,20275657,19612526,34375635764,37587604715,31296013232,29856655199,418097,471846,408166,391696,1696.15,NaN,NaN,NaN
1128,2024-06-25,1671.10,1716.95,1671.10,1707.00,1711.35,1705.04,1757.5,1363.55,37260774,6.353113e+10,690683,20.759,45.85,1,0,0,2,25,6,2024,1,177,2,1,1654.49,1602.03,1547.77,1525.30,34.66,36.76,30.32,27.48,1648.16,1599.02,1545.04,1524.34,24585251,23796383,21066896,19759836,40726192837,38026814031,32761367259,30237999548,476584,478943,415756,394555,1683.80,NaN,NaN,NaN
1129,2024-06-26,1707.95,1710.90,1691.10,1693.50,1701.50,1703.01,1757.5,1363.55,23798333,4.052887e+10,424591,20.639,19.80,0,0,0,1,26,6,2024,2,178,2,1,1669.43,1616.58,1555.79,1529.64,34.44,31.08,30.48,27.37,1664.00,1612.23,1553.15,1528.78,26160688,22398564,21432422,19638403,43611478080,36237510854,33487986023,30169390436,491500,454808,419377,394760,NaN,NaN,NaN,NaN
1130,2024-06-27,1696.10,1713.00,1686.60,1696.45,1696.15,1697.92,1757.5,1363.55,27537325,4.675614e+10,586504,20.574,26.40,0,0,0,2,27,6,2024,3,179,2,1,1682.05,1626.21,1564.38,1533.78,35.21,27.69,30.56,27.48,1677.42,1623.84,1561.65,1532.89,27779183,21513331,21701466,19634090,46576916482,35208163531,34110183335,30276181845,524781,450541,425809,398354,NaN,NaN,NaN,NaN


## Data standardization

### Stock price based columns

In [21]:
stock_price_cols = [
    'Open', 'High', 'Low', 'LTP', '52W H', '52W L'
] + (
    stock_df.filter(regex = "Close.*").columns.to_list() +
    stock_df.filter(regex = "Range.*").columns.to_list() +
    stock_df.filter(regex = "VWAP.*").columns.to_list() +
    stock_df.filter(regex = "Target.*").columns.to_list()
)

stock_price_cols

['Open',
 'High',
 'Low',
 'LTP',
 '52W H',
 '52W L',
 'Close',
 'Close 7MA',
 'Close 15MA',
 'Close 30MA',
 'Close 60MA',
 'Range',
 'Range 7MA',
 'Range 15MA',
 'Range 30MA',
 'Range 60MA',
 'VWAP',
 'VWAP 7MA',
 'VWAP 15MA',
 'VWAP 30MA',
 'VWAP 60MA',
 'Target 3D',
 'Target 7D',
 'Target 15D',
 'Target 30D']

In [22]:
stock_df[stock_price_cols] = stock_df[stock_price_cols].div(stock_df['Close'], axis = 0).round(3)
stock_df[stock_price_cols]

,Open,High,Low,LTP,52W H,52W L,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.998,1.001,0.994,1.000,1.958,0.848,1.0,1.000,1.000,1.000,1.000,0.007,0.007,0.007,0.007,0.007,0.998,0.998,0.998,0.998,0.998,0.971,1.003,0.970,0.970
1,0.994,1.001,0.994,0.999,1.945,0.842,1.0,0.997,0.997,0.997,0.997,0.007,0.007,0.007,0.007,0.007,0.998,0.995,0.995,0.995,0.995,0.980,0.999,0.967,0.971
2,1.011,1.013,0.996,1.000,1.974,0.855,1.0,1.008,1.008,1.008,1.008,0.017,0.010,0.010,0.010,0.010,1.002,1.007,1.007,1.007,1.007,0.991,1.017,0.981,0.979
3,1.015,1.017,0.996,0.999,2.017,0.874,1.0,1.022,1.022,1.022,1.022,0.021,0.013,0.013,0.013,0.013,1.005,1.023,1.023,1.023,1.023,1.025,1.035,0.978,0.983
4,0.999,1.009,0.993,1.000,1.986,0.860,1.0,1.005,1.005,1.005,1.005,0.015,0.013,0.013,0.013,0.013,1.001,1.006,1.006,1.006,1.006,1.018,1.021,0.970,0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,0.985,1.002,0.984,0.999,1.051,0.815,1.0,0.978,0.952,0.920,0.909,0.018,0.018,0.022,0.018,0.016,0.996,0.975,0.951,0.919,0.909,1.014,NaN,NaN,NaN
1128,0.976,1.003,0.976,0.997,1.027,0.797,1.0,0.967,0.936,0.904,0.891,0.027,0.020,0.021,0.018,0.016,0.996,0.963,0.934,0.903,0.891,0.984,NaN,NaN,NaN
1129,1.004,1.006,0.994,0.995,1.033,0.801,1.0,0.981,0.950,0.914,0.899,0.012,0.020,0.018,0.018,0.016,1.001,0.978,0.948,0.913,0.898,NaN,NaN,NaN,NaN
1130,1.000,1.010,0.994,1.000,1.036,0.804,1.0,0.992,0.959,0.922,0.904,0.016,0.021,0.016,0.018,0.016,1.001,0.989,0.957,0.921,0.904,NaN,NaN,NaN,NaN


### Volume based columns

In [23]:
volume_cols = stock_df.filter(regex = "Volume.*").columns.to_list()

volume_cols

['Volume', 'Volume 7MA', 'Volume 15MA', 'Volume 30MA', 'Volume 60MA']

In [24]:
stock_df[volume_cols] = stock_df[volume_cols].div(stock_df['Volume'], axis = 0).round(3)
stock_df[volume_cols]

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.799,0.799,0.799,0.799
2,1.0,0.635,0.635,0.635,0.635
3,1.0,0.724,0.724,0.724,0.724
4,1.0,0.629,0.629,0.629,0.629
...,...,...,...,...,...
1127,1.0,1.470,1.663,1.421,1.374
1128,1.0,0.660,0.639,0.565,0.530
1129,1.0,1.099,0.941,0.901,0.825
1130,1.0,1.009,0.781,0.788,0.713


### Value based columns

In [25]:
value_cols = stock_df.filter(regex = "Value.*").columns.to_list()

value_cols

['Value', 'Value 7MA', 'Value 15MA', 'Value 30MA', 'Value 60MA']

In [26]:
stock_df[value_cols] = stock_df[value_cols].div(stock_df['Value'], axis = 0).round(3)
stock_df[value_cols]

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.797,0.797,0.797,0.797
2,1.0,0.637,0.637,0.637,0.637
3,1.0,0.735,0.735,0.735,0.735
4,1.0,0.630,0.630,0.630,0.630
...,...,...,...,...,...
1127,1.0,1.447,1.582,1.317,1.256
1128,1.0,0.641,0.599,0.516,0.476
1129,1.0,1.076,0.894,0.826,0.744
1130,1.0,0.996,0.753,0.730,0.648


### Trade count based columns

In [27]:
trade_count_cols = stock_df.filter(regex = "#Trades.*").columns.to_list()

trade_count_cols

['#Trades', '#Trades 7MA', '#Trades 15MA', '#Trades 30MA', '#Trades 60MA']

In [28]:
stock_df[trade_count_cols] = stock_df[trade_count_cols].div(stock_df['#Trades'], axis = 0).round(3)
stock_df[trade_count_cols]

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.723,0.723,0.723,0.723
2,1.0,0.654,0.654,0.654,0.654
3,1.0,0.747,0.747,0.747,0.747
4,1.0,0.690,0.690,0.690,0.690
...,...,...,...,...,...
1127,1.0,1.079,1.217,1.053,1.011
1128,1.0,0.690,0.693,0.602,0.571
1129,1.0,1.158,1.071,0.988,0.930
1130,1.0,0.895,0.768,0.726,0.679


### Dropping unnecessary columns

In [29]:
stock_df = stock_df.drop(columns = ['Date', 'Close', 'Range', 'Volume', 'Value', '#Trades'])
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,PE,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.998,1.001,0.994,1.000,0.998,1.958,0.848,31.185,1,0,0,1,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.007,0.007,0.007,0.007,0.998,0.998,0.998,0.998,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.971,1.003,0.970,0.970
1,0.994,1.001,0.994,0.999,0.998,1.945,0.842,31.384,1,0,0,2,2,1,2020,3,2,1,1,0.997,0.997,0.997,0.997,0.007,0.007,0.007,0.007,0.995,0.995,0.995,0.995,0.799,0.799,0.799,0.799,0.797,0.797,0.797,0.797,0.723,0.723,0.723,0.723,0.980,0.999,0.967,0.971
2,1.011,1.013,0.996,1.000,1.002,1.974,0.855,30.937,0,0,0,1,3,1,2020,4,3,1,1,1.008,1.008,1.008,1.008,0.010,0.010,0.010,0.010,1.007,1.007,1.007,1.007,0.635,0.635,0.635,0.635,0.637,0.637,0.637,0.637,0.654,0.654,0.654,0.654,0.991,1.017,0.981,0.979
3,1.015,1.017,0.996,0.999,1.005,2.017,0.874,30.267,0,0,0,2,6,1,2020,0,6,1,3,1.022,1.022,1.022,1.022,0.013,0.013,0.013,0.013,1.023,1.023,1.023,1.023,0.724,0.724,0.724,0.724,0.735,0.735,0.735,0.735,0.747,0.747,0.747,0.747,1.025,1.035,0.978,0.983
4,0.999,1.009,0.993,1.000,1.001,1.986,0.860,30.746,1,0,0,1,7,1,2020,1,7,1,1,1.005,1.005,1.005,1.005,0.013,0.013,0.013,0.013,1.006,1.006,1.006,1.006,0.629,0.629,0.629,0.629,0.630,0.630,0.630,0.630,0.690,0.690,0.690,0.690,1.018,1.021,0.970,0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,0.985,1.002,0.984,0.999,0.996,1.051,0.815,20.286,1,0,0,1,24,6,2024,0,176,2,3,0.978,0.952,0.920,0.909,0.018,0.022,0.018,0.016,0.975,0.951,0.919,0.909,1.470,1.663,1.421,1.374,1.447,1.582,1.317,1.256,1.079,1.217,1.053,1.011,1.014,NaN,NaN,NaN
1128,0.976,1.003,0.976,0.997,0.996,1.027,0.797,20.759,1,0,0,2,25,6,2024,1,177,2,1,0.967,0.936,0.904,0.891,0.020,0.021,0.018,0.016,0.963,0.934,0.903,0.891,0.660,0.639,0.565,0.530,0.641,0.599,0.516,0.476,0.690,0.693,0.602,0.571,0.984,NaN,NaN,NaN
1129,1.004,1.006,0.994,0.995,1.001,1.033,0.801,20.639,0,0,0,1,26,6,2024,2,178,2,1,0.981,0.950,0.914,0.899,0.020,0.018,0.018,0.016,0.978,0.948,0.913,0.898,1.099,0.941,0.901,0.825,1.076,0.894,0.826,0.744,1.158,1.071,0.988,0.930,NaN,NaN,NaN,NaN
1130,1.000,1.010,0.994,1.000,1.001,1.036,0.804,20.574,0,0,0,2,27,6,2024,3,179,2,1,0.992,0.959,0.922,0.904,0.021,0.016,0.018,0.016,0.989,0.957,0.921,0.904,1.009,0.781,0.788,0.713,0.996,0.753,0.730,0.648,0.895,0.768,0.726,0.679,NaN,NaN,NaN,NaN


### Saving standardized data

In [30]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-standardized.parquet'), 
    index = False
)
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,PE,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.998,1.001,0.994,1.000,0.998,1.958,0.848,31.185,1,0,0,1,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.007,0.007,0.007,0.007,0.998,0.998,0.998,0.998,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.971,1.003,0.970,0.970
1,0.994,1.001,0.994,0.999,0.998,1.945,0.842,31.384,1,0,0,2,2,1,2020,3,2,1,1,0.997,0.997,0.997,0.997,0.007,0.007,0.007,0.007,0.995,0.995,0.995,0.995,0.799,0.799,0.799,0.799,0.797,0.797,0.797,0.797,0.723,0.723,0.723,0.723,0.980,0.999,0.967,0.971
2,1.011,1.013,0.996,1.000,1.002,1.974,0.855,30.937,0,0,0,1,3,1,2020,4,3,1,1,1.008,1.008,1.008,1.008,0.010,0.010,0.010,0.010,1.007,1.007,1.007,1.007,0.635,0.635,0.635,0.635,0.637,0.637,0.637,0.637,0.654,0.654,0.654,0.654,0.991,1.017,0.981,0.979
3,1.015,1.017,0.996,0.999,1.005,2.017,0.874,30.267,0,0,0,2,6,1,2020,0,6,1,3,1.022,1.022,1.022,1.022,0.013,0.013,0.013,0.013,1.023,1.023,1.023,1.023,0.724,0.724,0.724,0.724,0.735,0.735,0.735,0.735,0.747,0.747,0.747,0.747,1.025,1.035,0.978,0.983
4,0.999,1.009,0.993,1.000,1.001,1.986,0.860,30.746,1,0,0,1,7,1,2020,1,7,1,1,1.005,1.005,1.005,1.005,0.013,0.013,0.013,0.013,1.006,1.006,1.006,1.006,0.629,0.629,0.629,0.629,0.630,0.630,0.630,0.630,0.690,0.690,0.690,0.690,1.018,1.021,0.970,0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,0.985,1.002,0.984,0.999,0.996,1.051,0.815,20.286,1,0,0,1,24,6,2024,0,176,2,3,0.978,0.952,0.920,0.909,0.018,0.022,0.018,0.016,0.975,0.951,0.919,0.909,1.470,1.663,1.421,1.374,1.447,1.582,1.317,1.256,1.079,1.217,1.053,1.011,1.014,NaN,NaN,NaN
1128,0.976,1.003,0.976,0.997,0.996,1.027,0.797,20.759,1,0,0,2,25,6,2024,1,177,2,1,0.967,0.936,0.904,0.891,0.020,0.021,0.018,0.016,0.963,0.934,0.903,0.891,0.660,0.639,0.565,0.530,0.641,0.599,0.516,0.476,0.690,0.693,0.602,0.571,0.984,NaN,NaN,NaN
1129,1.004,1.006,0.994,0.995,1.001,1.033,0.801,20.639,0,0,0,1,26,6,2024,2,178,2,1,0.981,0.950,0.914,0.899,0.020,0.018,0.018,0.016,0.978,0.948,0.913,0.898,1.099,0.941,0.901,0.825,1.076,0.894,0.826,0.744,1.158,1.071,0.988,0.930,NaN,NaN,NaN,NaN
1130,1.000,1.010,0.994,1.000,1.001,1.036,0.804,20.574,0,0,0,2,27,6,2024,3,179,2,1,0.992,0.959,0.922,0.904,0.021,0.016,0.018,0.016,0.989,0.957,0.921,0.904,1.009,0.781,0.788,0.713,0.996,0.753,0.730,0.648,0.895,0.768,0.726,0.679,NaN,NaN,NaN,NaN
